'''
A star grid planner

See Wikipedia article (https://en.wikipedia.org/wiki/A*_search_algorithm)
See Youtube (https://www.youtube.com/watch?v=icZj67PTFhc)
'''

import math
import matplotlib.pyplot as plt

show_animation = True
fig_min_x = -10
fig_min_y = -10
fig_max_x = 60
fig_max_y = 60

class AStarPlanner:
    def __init__(self, ox, oy, resolution, rr):
        """
        Initialize grid map
        ox: x position list of obstaclesL
        oy: y position list of obstacles
        resolution: grid resolution
        rr: robot radius
        """
        self.resolution = resolution
        self.rr = rr
        self.min_x, self.max_x = 0, 0
        self.min_y, self.max_y = 0, 0
        self.obstacle_map = None
        self.x_width, self.y_width = 0, 0
        self.motion = self.get_motion_model()
        self.calc_obstacle_map(ox, oy)

    class Node:
        def __init__(self, x, y, cost, parent_index):
            """
            Initialize a node
            """
            self.x = x
            self.y = y
            self.cost = cost
            self.parent_index = parent_index

        def __str__(self):
            return "Node info: " + str(self.x) + "," + str(self.y) + "," + str(self.cost) + "," + str(self.parent_index)

    def planning(self, sx, sy, gx, gy):
        """
        A star path search

        input:
            s_x: start x position [m]
            s_y: start y position [m]
            gx: goal x position [m]
            gy: goal y position [m]

        output:
            rx: x position list of the final path
            ry: y position list of the final path
        """

        start_node = self.Node(self.calc_xy_index(sx, self.min_x),
                               self.calc_xy_index(sy, self.min_y), 0.0, -1)
        goal_node = self.Node(self.calc_xy_index(gx, self.min_x),
                              self.calc_xy_index(gy, self.min_y), 0.0, -1)

        open_set, closed_set = dict(), dict()
        open_set[self.calc_grid_index(start_node)] = start_node

        while 1:
            if len(open_set) == 0:
                print("Open set is empty..")
                break

            c_id = min(open_set, key=lambda o: open_set[o].cost + self.calc_heuristic(goal_node, open_set[o]))
            current = open_set[c_id]

            # show graph
            if show_animation:  # pragma: no cover
                plt.plot(self.calc_grid_position(current.x, self.min_x),
                         self.calc_grid_position(current.y, self.min_y), "xc")
                plt.grid(True)
                plt.xlim(fig_min_x, fig_max_x)
                plt.ylim(fig_min_y, fig_max_y)
                # for stopping simulation with the esc key.
                plt.gcf().canvas.mpl_connect('key_release_event',
                                             lambda event: [exit(
                                                 0) if event.key == 'escape' else None])
                if len(closed_set.keys()) % 10 == 0:
                    plt.pause(0.001)

            if current.x == goal_node.x and current.y == goal_node.y:
                print("Find goal")
                goal_node.parent_index = current.parent_index
                goal_node.cost = current.cost
                break

            # Remove the item from the open set
            del open_set[c_id]

            # Add it to the closed set
            closed_set[c_id] = current

            # expand_grid search grid based on motion model
            for i, _ in enumerate(self.motion):
                node = self.Node(current.x + self.motion[i][0],
                                 current.y + self.motion[i][1],
                                 current.cost + self.motion[i][2], c_id)
                n_id = self.calc_grid_index(node)

                # If the node is not safe, do nothing
                if not self.verify_node(node):
                    continue

                if n_id in closed_set:
                    continue

                if n_id not in open_set:
                    open_set[n_id] = node  # discovered a new node
                else:
                    if open_set[n_id].cost > node.cost:
                        # This path is the best until now. record it
                        open_set[n_id] = node

        rx, ry = self.calc_final_path(goal_node, closed_set)

        return rx, ry

    def calc_final_path(self, goal_node, closed_set):
        # generate final course
        rx, ry = [self.calc_grid_position(goal_node.x, self.min_x)], [self.calc_grid_position(goal_node.y, self.min_y)]
        parent_index = goal_node.parent_index
        while parent_index != -1:
            n = closed_set[parent_index]
            rx.append(self.calc_grid_position(n.x, self.min_x))
            ry.append(self.calc_grid_position(n.y, self.min_y))
            parent_index = n.parent_index

        return rx, ry

    def calc_xy_index(self, position, min_pos):
        """
        calc xy index

        input: 
            position: unrounded position
            min_pos: min position value
        return:
            position in grid
        """
        return round((position-min_pos) / self.resolution)

    def calc_grid_index(self, node):
        return (node.y - self.min_y) * self.x_width + (node.x - self.min_x)

    def calc_grid_position(self, index, min_pos):
        """
        calc grid position
        
        input:
            index: index of node
            min_pos: min position value
        return:
            position in grid
        """
        return index * self.resolution + min_pos

    @staticmethod
    def get_motion_model():
        # dx, dy, cost
        return [[1, 0, 1], 
                [0, 1, 1],
                [-1, 0, 1],
                [0, -1, 1],
                [-1, -1, math.sqrt(2)],
                [-1, 1, math.sqrt(2)],
                [1, -1, math.sqrt(2)],
                [1, 1, math.sqrt(2)]]
        
    @staticmethod
    def calc_heuristic(n1, n2):
        return math.hypot(n1.x - n2.x, n1.y - n2.y)

    def verify_node(self, node):
        px = self.calc_grid_position(node.x, self.min_x)
        py = self.calc_grid_position(node.y, self.min_y)

        if px < self.min_x:
            return False
        elif py < self.min_y:
            return False
        elif px >= self.max_x:
            return False
        elif py >= self.max_y:
            return False

        # collision check
        if self.obstacle_map[node.x][node.y]:
            return False

        return True
    
    def calc_obstacle_map(self, ox, oy):
        self.min_x = round(min(ox))
        self.min_y = round(min(oy))
        self.max_x = round(max(ox))
        self.max_y = round(max(oy))
        print("min_x:", self.min_x)
        print("min_y:", self.min_y)
        print("max_x:", self.max_x)
        print("max_y:", self.max_y)

        self.x_width = round((self.max_x - self.min_x) / self.resolution)
        self.y_width = round((self.max_y - self.min_y) / self.resolution)
        print("x_width:", self.x_width)
        print("y_width:", self.y_width)

        # obstacle map generation
        self.obstacle_map = [[False for _ in range(self.y_width)]
                             for _ in range(self.x_width)]
        for ix in range(self.x_width):
            x = self.calc_grid_position(ix, self.min_x)
            for iy in range(self.y_width):
                y = self.calc_grid_position(iy, self.min_y)
                for iox, ioy in zip(ox, oy):
                    d = math.hypot(iox - x, ioy - y)
                    if d <= self.rr:
                        self.obstacle_map[ix][iy] = True
                        break

def main():
    print("Start!!")

    # start and goal position
    sx = 10.0  # [m]
    sy = 10.0  # [m]
    gx = 50.0  # [m]
    gy = 50.0  # [m]
    grid_size = 2.0  # [m]
    robot_radius = 1.0  # [m]

    # set obstacle positions
    ox, oy = [], []
    for i in range(-10, 60):
        ox.append(i)
        oy.append(-10.0)
    for i in range(-10, 60):
        ox.append(60.0)
        oy.append(i)
    for i in range(-10, 61):
        ox.append(i)
        oy.append(60.0)
    for i in range(-10, 61):
        ox.append(-10.0)
        oy.append(i)
    for i in range(-10, 40):
        ox.append(20.0)
        oy.append(i)
    for i in range(0, 40):
        ox.append(40.0)
        oy.append(60.0 - i)
        
    # check obstacles
    if (min(ox) < fig_min_x) or (min(oy) < fig_min_y) or (max(ox) > fig_max_x) or (max(oy) > fig_max_y):
        print("Obstacle out of bounds")
        exit()

    if show_animation:  # pragma: no cover
        plt.plot(ox, oy, ".k")
        plt.plot(sx, sy, "og")
        plt.plot(gx, gy, "xb")
        plt.grid(True)
        plt.xlim(fig_min_x, fig_max_x)
        plt.ylim(fig_min_y, fig_max_y)

    a_star = AStarPlanner(ox, oy, grid_size, robot_radius)
    rx, ry = a_star.planning(sx, sy, gx, gy)

    if show_animation:  # pragma: no cover
        plt.plot(ox, oy, ".k")
        plt.plot(rx, ry, "-r")
        plt.grid(True)
        plt.xlim(fig_min_x, fig_max_x)
        plt.ylim(fig_min_y, fig_max_y)
        plt.pause(0.001)
        plt.show()


if __name__ == '__main__':
    main()

### Probabilistic Roadmaps
1. 

### Optimization based appoaches
1. Reactive control - potential-based methods
2. Optimization based methods - elastic bands, CHOMP (STOMP, ITOMP) and TrajOpt
3. General nonlinear optimization strategy: 
$$ min_{\theta_1...\theta_T} \sum_t \vert \vert \theta_{t+1} - \theta_{t}\vert \vert+other~costs$$
$$s.t. \theta_0 = start~state, \theta_t = goal~state$$
$$satisfy~joint~limits$$
$$no~collision$$
4. Despite the first few constraints being linear/convex, the last requirement on collision can be non-convex. To handle this non-convexity, we can use sequential convex optimization to approximate the problem locally with a convex optimization problem.
5. Typically, collision avoidance is added via a penalty-based function using the signed distance function using a hinge loss function.

#### Continuous time safety
1. Collision check against swept-out volume. This is faster
2. 

#### Sequential convex optimization
